In [2]:
import os, json
from pymongo import MongoClient
from dotenv import load_dotenv
from bson.decimal128 import Decimal128
from datetime import datetime, timezone

load_dotenv()
client = MongoClient(os.getenv("MONGODB_URI"), tz_aware=True)
db = client[os.getenv("MONGODB_DATABASE")]
dispensers = db["dispensers"]

location = "loboc"

pipeline = [
    {"$match": {"location": location}},
    {
        "$lookup": {
            "from": "pumps",
            "let": {"did": "$id", "loc": "$location"},
            "pipeline": [
                {
                    "$match": {
                        "$expr": {
                            "$and": [
                                {"$eq": [{"$toString": "$dispenser_id"}, {"$toString": "$$did"}]},
                                {"$eq": ["$location", "$$loc"]}
                            ]
                        }
                    }
                },
                {"$project": {"_id": 0, "id": 1, "name": 1}}
            ],
            "as": "pumps"
        }
    },
    {"$unwind": {"path": "$pumps", "preserveNullAndEmptyArrays": False}},
    {
        "$lookup": {
            "from": "pump_inventories",
            "let": {"pid": {"$toString": "$pumps.id"}, "loc": "$location"},
            "pipeline": [
                {
                    "$match": {
                        "$expr": {
                            "$and": [
                                {"$eq": [{"$toString": "$pump_id"}, "$$pid"]},
                                {"$eq": ["$location", "$$loc"]}
                            ]
                        }
                    }
                },
                {"$sort": {"date": -1, "created_at": -1}},
                {"$limit": 1},
                {"$project": {"_id": 0}}
            ],
            "as": "latest"
        }
    },
    {"$addFields": {"pumps.latest_inventory": {"$arrayElemAt": ["$latest", 0]}}},
    {"$project": {"latest": 0}},
    {
        "$group": {
            "_id": "$id",
            "id": {"$first": "$id"},
            "name": {"$first": "$name"},
            "location": {"$first": "$location"},
            "pumps": {"$push": "$pumps"}
        }
    },
    {"$project": {"_id": 0, "id": 1, "name": 1, "location": 1, "pumps": 1}},
    {"$sort": {"name": 1}}
]

docs = list(dispensers.aggregate(pipeline, allowDiskUse=True))

def _enc(o):
    if isinstance(o, Decimal128):
        return str(o.to_decimal())
    if isinstance(o, datetime):
        return o.astimezone(timezone.utc).isoformat().replace("+00:00","Z")
    return str(o)

print(json.dumps(docs, indent=2, ensure_ascii=False, default=_enc))


[
  {
    "id": "4",
    "name": "dispenser-four",
    "location": "loboc",
    "pumps": [
      {
        "id": 7,
        "name": "regular",
        "latest_inventory": {
          "id": "0e726124-abf7-4ebc-9206-77344f0c34f2",
          "created_at": "2025-09-27T19:26:35.056000Z",
          "location": "loboc",
          "date": "2025-09-27T16:00:00Z",
          "dispenser_name": "dispenser-four",
          "pump_id": "7",
          "pump_name": "regular",
          "product": "regular",
          "unit": "liter",
          "price": "62.50",
          "beginning_inventory": "5000.00",
          "calibration": "0.00",
          "po": "0.00",
          "cash": "0.00",
          "ending_inventory": "5000.00",
          "starting_liter_meter": "26309.00",
          "ending_liter_meter": "26309.00"
        }
      },
      {
        "id": 8,
        "name": "diesel",
        "latest_inventory": {
          "id": "d8f37a79-c974-45be-b7ad-0736177f1d11",
          "created_at": "2025-09-27T1